# ETL Process for Hockey Data

###### Install/Import Dependencies

In [1]:
# # install API dependency (if needed)
# pip install kaggle

In [2]:
# import database connection details
from config import user, password, host, database, param_dic

# import dependencies
import kaggle
import numpy as np
import os
import pandas as pd
import psycopg2
import psycopg2.extras as extras
from sqlalchemy import create_engine
import time

In [3]:
# record time for start for total run time
nb_start = time.time()

###### Extract Data from Source

In [4]:
# Go to https://www.kaggle.com/user_name/account to create API token
# Place kaggle.json (download) into 'C:\Users\computer_name\.kaggle\kaggle.json'
kaggle.api.authenticate()

# download kaggle dataset (NHL Hockey Data)
# will overwrite local files, if they exist
kaggle.api.dataset_download_files(
    'martinellis/nhl-game-data',
    path = '../project-2-zamboni-zealots/data/',
    unzip = True
)

###### Import CSVs to Dataframes, Transform

In [5]:
# read player_info csv to df
team_info_df = pd.read_csv('data/team_info.csv')
team_info_df.head(5)

,team_id,franchiseId,shortName,teamName,abbreviation,link
0,1,23,New Jersey,Devils,NJD,/api/v1/teams/1
1,4,16,Philadelphia,Flyers,PHI,/api/v1/teams/4
2,26,14,Los Angeles,Kings,LAK,/api/v1/teams/26
3,14,31,Tampa Bay,Lightning,TBL,/api/v1/teams/14
4,6,6,Boston,Bruins,BOS,/api/v1/teams/6


In [6]:
# read game csv to df
game_df = pd.read_csv('data/game.csv')

# drop duplicate rows, keep first record
game_df.drop_duplicates(
    subset = 'game_id',
    keep = 'first',
    inplace = True
)

# return first 5 rows
game_df.head(5)

,game_id,season,type,date_time_GMT,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz
0,2016020045,20162017,R,2016-10-19T00:30:00Z,4,16,4,7,home win REG,right,United Center,/api/v1/venues/null,America/Chicago,-5,CDT
1,2017020812,20172018,R,2018-02-07T00:00:00Z,24,7,4,3,away win OT,left,KeyBank Center,/api/v1/venues/null,America/New_York,-4,EDT
2,2015020314,20152016,R,2015-11-24T01:00:00Z,21,52,4,1,away win REG,right,MTS Centre,/api/v1/venues/null,America/Winnipeg,-5,CDT
3,2015020849,20152016,R,2016-02-17T00:00:00Z,52,12,1,2,home win REG,right,PNC Arena,/api/v1/venues/null,America/New_York,-4,EDT
4,2017020586,20172018,R,2017-12-30T03:00:00Z,20,24,1,2,home win REG,left,Honda Center,/api/v1/venues/null,America/Los_Angeles,-7,PDT


In [7]:
player_info_df = pd.read_csv('data/player_info.csv')
player_info_df.head(5)

,player_id,firstName,lastName,nationality,birthCity,primaryPosition,birthDate,birthStateProvince,height,height_cm,weight,shootsCatches
0,8466148,Marian,Hossa,SVK,Stará Lubovna,RW,1979-01-12 00:00:00,NaN,"6' 1""",185.42,207.0,L
1,8465058,Michal,Rozsival,CZE,Vlasim,D,1978-09-03 01:00:00,NaN,"6' 1""",185.42,210.0,R
2,8476906,Shayne,Gostisbehere,USA,Pembroke Pines,D,1993-04-20 01:00:00,FL,"5' 11""",180.34,180.0,L
3,8466285,Brian,Campbell,CAN,Strathroy,D,1979-05-23 01:00:00,ON,"5' 10""",177.80,192.0,L
4,8470607,Brent,Seabrook,CAN,Richmond,D,1985-04-20 01:00:00,BC,"6' 3""",190.50,220.0,R


In [8]:
# read game_plays csv to df
game_plays_df = pd.read_csv('data/game_plays.csv')

# drop duplicate rows, keep last record
game_plays_df.drop_duplicates(
    subset = 'play_id',
    keep = 'last',
    inplace = True
)

# return first 5 rows
game_plays_df.head(5)

,play_id,game_id,team_id_for,team_id_against,event,secondaryType,x,y,period,periodType,periodTime,periodTimeRemaining,dateTime,goals_away,goals_home,description,st_x,st_y
0,2016020045_1,2016020045,NaN,NaN,Game Scheduled,NaN,NaN,NaN,1,REGULAR,0,1200.0,2016-10-18 23:40:58,0,0,Game Scheduled,NaN,NaN
1,2016020045_2,2016020045,NaN,NaN,Period Ready,NaN,NaN,NaN,1,REGULAR,0,1200.0,2016-10-19 01:35:28,0,0,Period Ready,NaN,NaN
2,2016020045_3,2016020045,NaN,NaN,Period Start,NaN,NaN,NaN,1,REGULAR,0,1200.0,2016-10-19 01:40:50,0,0,Period Start,NaN,NaN
3,2016020045_4,2016020045,16.0,4.0,Faceoff,NaN,0.0,0.0,1,REGULAR,0,1200.0,2016-10-19 01:40:50,0,0,Jonathan Toews faceoff won against Claude Giroux,0.0,0.0
4,2016020045_5,2016020045,16.0,4.0,Shot,Wrist Shot,-71.0,9.0,1,REGULAR,54,1146.0,2016-10-19 01:41:44,0,0,Artem Anisimov Wrist Shot saved by Michal Neuv...,71.0,-9.0


In [9]:
# read game_teams_stats csv to df
game_teams_stats_df = pd.read_csv('data/game_teams_stats.csv')
game_teams_stats_df.head(5)

,game_id,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways,blocked,startRinkSide
0,2016020045,4,away,False,REG,Dave Hakstol,4.0,27.0,30.0,6.0,4.0,2.0,50.9,12.0,9.0,11.0,left
1,2016020045,16,home,True,REG,Joel Quenneville,7.0,28.0,20.0,8.0,3.0,2.0,49.1,16.0,8.0,9.0,left
2,2017020812,24,away,True,OT,Randy Carlyle,4.0,34.0,16.0,6.0,3.0,1.0,43.8,7.0,4.0,14.0,right
3,2017020812,7,home,False,OT,Phil Housley,3.0,33.0,17.0,8.0,2.0,1.0,56.2,5.0,6.0,14.0,right
4,2015020314,21,away,True,REG,Patrick Roy,4.0,29.0,17.0,9.0,3.0,1.0,45.7,13.0,5.0,20.0,left


In [10]:
# load game_shifts csv to df
game_scratches_df = pd.read_csv('data/game_scratches.csv')
game_scratches_df.head()

,game_id,team_id,player_id
0,2016020045,16,8477845
1,2016020045,16,8477451
2,2016020045,16,8465058
3,2016020045,4,8476393
4,2016020045,4,8475462


In [11]:
# load game_officials csv to df
game_officials_df = pd.read_csv('data/game_officials.csv')
game_officials_df.head()

,game_id,official_name,official_type
0,2016020045,Dan O'Rourke,Referee
1,2016020045,Trevor Hanson,Referee
2,2016020045,Scott Driscoll,Linesman
3,2016020045,Lonnie Cameron,Linesman
4,2017020812,Justin St. Pierre,Referee


In [12]:
# load game_shifts csv to df
game_shifts_df = pd.read_csv('data/game_shifts.csv')
game_shifts_df.head()

,game_id,player_id,period,shift_start,shift_end
0,2018020001,8466139,1,0,42.0
1,2018020001,8466139,1,207,247.0
2,2018020001,8466139,1,375,413.0
3,2018020001,8466139,1,556,574.0
4,2018020001,8466139,1,605,631.0


In [13]:
# load game_shifts csv to df
game_penalties_df = pd.read_csv('data/game_penalties.csv')
game_penalties_df.head(5)

,play_id,penaltySeverity,penaltyMinutes
0,2016020045_41,Minor,2
1,2016020045_101,Minor,2
2,2016020045_134,Minor,2
3,2016020045_174,Minor,2
4,2016020045_189,Minor,2


In [14]:
# load game_shifts csv to df
game_skater_stats_df = pd.read_csv('data/game_skater_stats.csv')
game_skater_stats_df.head(5)

,game_id,player_id,team_id,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,...,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,shortHandedTimeOnIce,powerPlayTimeOnIce
0,2016020045,8468513,4,955,1,0,0,2.0,0,0,...,0,1.0,1.0,0,0,1.0,1,858,97,0
1,2016020045,8476906,4,1396,1,0,4,2.0,0,0,...,0,1.0,2.0,0,0,2.0,0,1177,0,219
2,2016020045,8474668,4,915,0,0,1,1.0,0,0,...,0,2.0,0.0,0,0,0.0,-1,805,0,110
3,2016020045,8473512,4,1367,3,0,0,0.0,0,2,...,27,0.0,0.0,0,0,0.0,-1,1083,19,265
4,2016020045,8471762,4,676,0,0,3,2.0,0,0,...,0,0.0,1.0,0,0,0.0,-1,613,63,0


In [15]:
# read game_goalie_stats csv to df
game_goalie_stats_df = pd.read_csv('data/game_goalie_stats.csv')
game_goalie_stats_df.head(5)

,game_id,player_id,team_id,timeOnIce,assists,goals,pim,shots,saves,powerPlaySaves,shortHandedSaves,evenSaves,shortHandedShotsAgainst,evenShotsAgainst,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,evenStrengthSavePercentage
0,2016020045,8473607,4,1504,0,0,0,16,12,1,0,11,0,13,3,NaN,75.000000,33.333333,84.615385
1,2016020045,8473461,4,2011,0,0,0,11,9,1,0,8,0,10,1,L,81.818182,100.000000,80.000000
2,2016020045,8470645,16,3598,0,0,0,27,23,2,0,21,0,23,4,W,85.185185,50.000000,91.304348
3,2017020812,8468011,24,3696,0,0,0,33,30,1,2,27,3,28,2,W,90.909091,50.000000,96.428571
4,2017020812,8475215,7,3539,0,0,0,33,29,4,1,24,1,27,5,L,87.878788,80.000000,88.888889


In [16]:
# read game_goalie_stats csv to df"
game_plays_players_df = pd.read_csv('data/game_plays_players.csv')
game_plays_players_df.head(5)

,play_id,game_id,player_id,playerType
0,2016020045_4,2016020045,8473604,Winner
1,2016020045_4,2016020045,8473512,Loser
2,2016020045_5,2016020045,8473573,Shooter
3,2016020045_5,2016020045,8473607,Goalie
4,2016020045_6,2016020045,8474141,Scorer


In [17]:
# read game_goalie_stats csv to df"
# replace NA values with FALSE boolean
game_goals_df = pd.read_csv('data/game_goals.csv').fillna(False)
game_goals_df.head(5)

C:\Users\Overmind\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,play_id,strength,gameWinningGoal,emptyNet
0,2016020045_6,Even,False,False
1,2016020045_97,Even,False,False
2,2016020045_103,Power Play,False,False
3,2016020045_140,Power Play,False,False
4,2016020045_197,Power Play,False,False


###### Connections to PostgreSQL Database

In [18]:
# create connection function with status print
def connect(params):
    conn = None
    try:
        print(f'Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print(f'Connection successful!')
    return conn

In [19]:
# create connector
engine = create_engine(f'postgresql://{user}:{password}@{host}:5432/{database}')

# display tables in database
engine.table_names()

['game',
 'game_plays',
 'game_teams_stats',
 'game_scratches',
 'team_info',
 'player_info',
 'game_officials',
 'game_shifts',
 'game_penalties',
 'game_skater_stats',
 'game_goalie_stats',
 'game_plays_players',
 'game_goals']

In [20]:
# test database connection
conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful!


###### Define Functions for ETL Loads

* Use 'load_data' for large datasets - it runs substantially faster than pd.to_sql()
* The 'load_data' function will force column headers lowercase, so need to adjust schema accordingly
* Use 'to_sql' for small datasets - executes pd.to_sql(), where difference in performance is nominal compared to 'load_data'

In [21]:
# function for reading dataframe, loading to sql database with status print
def load_data(conn, df, table):
    start = time.time()
    print(f'---------------')
    print(f'Initializing')
    print(f'---------------')
    # create list of tupples from dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute insert
    query  = 'INSERT INTO %s(%s) VALUES %%s' % (table, cols)
    cursor = conn.cursor()
    try:
        print(f'Loading {table} data to the PostgreSQL database...')
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print(f'---------------')
        print(f'Error: %s' % error)
        print(f'---------------')
        print(f'Load failed.')
        conn.rollback()
        cursor.close()
        return 1
    end = time.time()
    elapsed = round(end - start, 1)
    print(f'Load successful!  Run Time: {elapsed} seconds')
    print(f'---------------')
    cursor.close()

In [22]:
# function to use pandas to_sql function with status print
def to_sql(df, table):
    start = time.time()
    print(f'---------------')
    print(f'Initializing')
    print(f'---------------')
    try:
        print(f'Loading {table} data to the PostgreSQL database...')
        # use pd.to_sql() to set load details
        df.to_sql(
            table, 
            con = engine, 
            schema = 'public', 
            if_exists = 'append',
            index = False
        )
    except (Exception, psycopg2.DatabaseError) as error:
        print(f'---------------')
        print(f'Error: %s' % error)
        print(f'---------------')
        print(f'Load failed.')
        return 1
    end = time.time()
    elapsed = round(end - start, 1)
    print(f'Load successful!  Run Time: {elapsed} seconds')
    print(f'---------------')

###### Load Dataframes into PostgreSQL Database

In [23]:
# load df to sql db table (df, table)
to_sql(team_info_df, 'team_info')

---------------
Initializing
---------------
Loading team_info data to the PostgreSQL database...
Load successful!  Run Time: 0.0 seconds
---------------


In [24]:
# load df to sql db table (df, table)
to_sql(game_df, 'game')

---------------
Initializing
---------------
Loading game data to the PostgreSQL database...
Load successful!  Run Time: 3.6 seconds
---------------


In [25]:
# load df to sql db table (df, table)
to_sql(player_info_df, 'player_info')

---------------
Initializing
---------------
Loading player_info data to the PostgreSQL database...
Load successful!  Run Time: 0.6 seconds
---------------


In [26]:
# load df to sql db table
load_data(conn, game_plays_df, 'game_plays')

---------------
Initializing
---------------
Loading game_plays data to the PostgreSQL database...
Load successful!  Run Time: 258.0 seconds
---------------


In [27]:
# load df to sql db table (df, table)
to_sql(game_teams_stats_df, 'game_teams_stats')

---------------
Initializing
---------------
Loading game_teams_stats data to the PostgreSQL database...
Load successful!  Run Time: 9.8 seconds
---------------


In [28]:
# load df to sql db table (df, table)
to_sql(game_scratches_df, 'game_scratches')

---------------
Initializing
---------------
Loading game_scratches data to the PostgreSQL database...
Load successful!  Run Time: 17.2 seconds
---------------


In [29]:
# load df to sql db table (df, table)
to_sql(game_officials_df, 'game_officials')

---------------
Initializing
---------------
Loading game_officials data to the PostgreSQL database...
Load successful!  Run Time: 12.0 seconds
---------------


In [30]:
# load df to sql db table
load_data(conn, game_shifts_df, 'game_shifts')

---------------
Initializing
---------------
Loading game_shifts data to the PostgreSQL database...
Load successful!  Run Time: 443.2 seconds
---------------


In [31]:
# load df to sql db table (df, table)
to_sql(game_penalties_df, 'game_penalties')

---------------
Initializing
---------------
Loading game_penalties data to the PostgreSQL database...
Load successful!  Run Time: 34.8 seconds
---------------


In [32]:
# load df to sql db table
load_data(conn, game_skater_stats_df, 'game_skater_stats')

---------------
Initializing
---------------
Loading game_skater_stats data to the PostgreSQL database...
Load successful!  Run Time: 89.0 seconds
---------------


In [33]:
# load df to sql db table (df, table)
to_sql(game_goalie_stats_df, 'game_goalie_stats')

---------------
Initializing
---------------
Loading game_goalie_stats data to the PostgreSQL database...
Load successful!  Run Time: 10.1 seconds
---------------


In [34]:
# load df to sql db table
load_data(conn, game_plays_players_df, 'game_plays_players')

---------------
Initializing
---------------
Loading game_plays_players data to the PostgreSQL database...
Load successful!  Run Time: 182.8 seconds
---------------


In [35]:
# load df to sql db table (df, table)
to_sql(game_goals_df, 'game_goals')

---------------
Initializing
---------------
Loading game_goals data to the PostgreSQL database...
Load successful!  Run Time: 23.6 seconds
---------------


In [37]:
# print total run time
nb_end = time.time()
nb_elapsed = round((nb_end - nb_start)/60, 1)
print(f'Completed ETL process!')
print(f'Total Run Time: {nb_elapsed} minutes')

Completed ETL process!
Total Run Time: 19.4 minutes
